LSTM

In [1]:
import pandas as pd
from keras.models import Sequential
from keras.layers import LSTM, Dense
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import joblib

# Carga tus datos desde un archivo CSV
data = pd.read_csv('appliance_usage_data2.csv')

# Preprocesamiento de datos
X = data[['Hora', 'Watts usados', 'Porcentaje de fallos']].values
y = data['Reporte de fallo'].values

# Normalizar los datos
scaler = MinMaxScaler(feature_range=(0, 1))
X = scaler.fit_transform(X)

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Cambiar la forma de los datos para LSTM (muestras, pasos de tiempo, características)
X_train = X_train.reshape((X_train.shape[0], 1, X_train.shape[1]))
X_test = X_test.reshape((X_test.shape[0], 1, X_test.shape[1]))

# Crear el modelo LSTM
model = Sequential()
model.add(LSTM(50, input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(Dense(1))
model.compile(loss='mae', optimizer='adam')

# Entrenar el modelo
history = model.fit(X_train, y_train, epochs=10, batch_size=72, validation_data=(X_test, y_test), verbose=2, shuffle=False)

# Guardando el modelo
model.save('lstm_model.h5')

# Guardando el escalador
joblib.dump(scaler, 'scaler_lstm.pkl')

# Evaluar el modelo
y_pred = model.predict(X_test)
print(y_pred)

# Mostrar la precisión del modelo
from sklearn.metrics import mean_squared_error
from math import sqrt
rmse = sqrt(mean_squared_error(y_test, y_pred))

print(f'RMSE: {rmse}')

Epoch 1/10
5556/5556 - 24s - loss: 0.5001 - val_loss: 0.4998 - 24s/epoch - 4ms/step
Epoch 2/10
5556/5556 - 21s - loss: 0.5001 - val_loss: 0.4997 - 21s/epoch - 4ms/step
Epoch 3/10
5556/5556 - 20s - loss: 0.5000 - val_loss: 0.4997 - 20s/epoch - 4ms/step
Epoch 4/10
5556/5556 - 20s - loss: 0.5000 - val_loss: 0.4997 - 20s/epoch - 4ms/step
Epoch 5/10
5556/5556 - 21s - loss: 0.4999 - val_loss: 0.4996 - 21s/epoch - 4ms/step
Epoch 6/10
5556/5556 - 19s - loss: 0.4999 - val_loss: 0.4996 - 19s/epoch - 3ms/step
Epoch 7/10
5556/5556 - 19s - loss: 0.4999 - val_loss: 0.4996 - 19s/epoch - 3ms/step
Epoch 8/10
5556/5556 - 19s - loss: 0.4999 - val_loss: 0.4996 - 19s/epoch - 3ms/step
Epoch 9/10
5556/5556 - 19s - loss: 0.4999 - val_loss: 0.4996 - 19s/epoch - 3ms/step
Epoch 10/10
5556/5556 - 19s - loss: 0.4999 - val_loss: 0.4996 - 19s/epoch - 3ms/step


c:\Users\sebas\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


3125/3125 [==============================] - 6s 2ms/step
[[0.04750502]
 [0.04729916]
 [0.04577195]
 ...
 [0.04599118]
 [0.04617105]
 [0.04649209]]
RMSE: 0.67464689310317


Haciendo predicciones

In [8]:
import pandas as pd
from keras.models import load_model
from sklearn.preprocessing import MinMaxScaler

# Carga tus datos desde un archivo CSV
data = pd.read_csv('appliance_usage_data1.csv')

# Selecciona las columnas que quieres usar como datos de entrada
input_data = data[['Hora', 'Watts usados', 'Porcentaje de fallos']].values

# Normalizar los datos
scaler = MinMaxScaler(feature_range=(0, 1))
input_data = scaler.fit_transform(input_data)

# Cambiar la forma de los datos para LSTM (muestras, pasos de tiempo, características)
input_data = input_data.reshape((input_data.shape[0], 1, input_data.shape[1]))

# Cargar el modelo
model = load_model('lstm_model.h5')

# Hacer una predicción
predictions = model.predict(input_data)

print(predictions)

15625/15625 [==============================] - 34s 2ms/step
[[0.0459889 ]
 [0.04747513]
 [0.04626952]
 ...
 [0.04733466]
 [0.04744442]
 [0.0463929 ]]
